In [85]:
import glob
import pandas as pd
from datetime import datetime
import json 
import xml.etree.ElementTree as ET

In [20]:
tempfile    = "dealership_temp.tmp"               # file used to store all extracted data
logfile    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile = "dealership_transformed_data.csv"   # file where transformed data is stored

In [10]:
data1= pd.read_csv('used_car_prices1.csv')

In [42]:
data1.head(11)

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.000000,Petrol
1,sx4,2013,7089.552239,Diesel
2,ciaz,2017,10820.895522,Petrol
3,wagon r,2011,4253.731343,Petrol
4,swift,2014,6865.671642,Diesel
5,vitara brezza,2018,13805.970149,Diesel
6,ciaz,2015,10074.626866,Petrol
7,s cross,2015,9701.492537,Diesel
8,ciaz,2016,13059.701493,Diesel
9,ciaz,2015,11119.402985,Diesel


In [14]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

In [18]:
extract_from_csv('used_car_prices1.csv')

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.000000,Petrol
1,sx4,2013,7089.552239,Diesel
2,ciaz,2017,10820.895522,Petrol
3,wagon r,2011,4253.731343,Petrol
4,swift,2014,6865.671642,Diesel
5,vitara brezza,2018,13805.970149,Diesel
6,ciaz,2015,10074.626866,Petrol
7,s cross,2015,9701.492537,Diesel
8,ciaz,2016,13059.701493,Diesel
9,ciaz,2015,11119.402985,Diesel


In [30]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

In [31]:
extract_from_json('used_car_prices1.json')

,car_model,year_of_manufacture,price,fuel
0,ritz,2012,4626.865672,Diesel
1,ritz,2011,3507.462687,Petrol
2,swift,2014,7388.059701,Diesel
3,ertiga,2014,8955.223881,Diesel
4,dzire,2014,8208.955224,Diesel
5,sx4,2011,4402.985075,CNG
6,dzire,2015,6940.298507,Petrol
7,800,2003,522.388060,Petrol
8,alto k10,2016,4477.611940,Petrol
9,sx4,2003,3358.208955,Petrol


In [68]:
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["car_model", "year_of_manufacture", "Price","fuel"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for x in root:
        car_model = x.find("car_model").text
        fuel = x.find("fuel").text
        year_of_manufacture = int(x.find("year_of_manufacture").text)
        Price = float(x.find("price").text)
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture,"fuel":fuel,"Price":Price}, ignore_index=True)
    return dataframe

In [69]:
extract_from_xml('used_car_prices1.xml')

,car_model,year_of_manufacture,Price,fuel
0,corolla altis,2013,10373.134328,Petrol
1,etios cross,2015,6716.417910,Petrol
2,fortuner,2014,27985.074627,Diesel
3,fortuner,2015,35074.626866,Diesel
4,fortuner,2017,49253.731343,Diesel
5,etios liva,2014,7089.552239,Diesel
6,innova,2017,29477.611940,Petrol
7,fortuner,2010,13805.970149,Diesel
8,corolla altis,2011,6492.537313,Petrol
9,corolla altis,2016,21268.656716,Petrol


In [71]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','Price','fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

In [77]:
def transform(data):
        data['Price'] = round(data.Price,2)
        return data

In [76]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  

In [83]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

In [86]:
log("ETL Job Started")
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data 
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")
log("ETL Job Ended")